# Práctica 1: Criptografía clásica

## Fundamentos de Criptografía y Seguridad Informática
## UAM, 2022/2023

### Maitane Gómez González
### Ana Martínez Sabiote

El lenguaje elegido para la implementación de esta práctica ha sido Python. La memoria de la práctica y el código en Python desarrollado están contenidos en este notebook. Se entrega tanto en formato .pdf para su lectura, como el fichero .ipynb, que permite la ejecución del código en Jupyter Notebook.

## 1. Sustitución monoalfabeto

## 1.a Método afín

Para implementar el método afín, al igual que para la mayoría de los demás cifrados de la práctica, utilizamos un alfabeto formato por todos los caracteres en minúsculas y todos los caracteres en mayúsculas. Por tanto, un alfabeto de 52 elementos. Si una cadena contiene además símbolos que no están contemplados en nuestro alfabeto, éstos no se tienen en cuenta. 

Para el desarrollo de este apartado hemos implementado las siguientes funciones
- La función *algoritmo_euclides(a,b)* que calcula el máximo común divisor de los dos números pasados como parámetros de manera recursiva.
- La función *algoritmo_euclides_extendido(a,b)* que aplica el algoritmo de extendido de Euclides que hemos estudiado. Esta función recibe dos números *a* y *b* como parámetros y devuelve el máximo común divisor de ellos, *u* y *v*, los coeficientes de la Identidad de Bézout: **1=a\*u + b\*v**. Por tanto con esta función obtenemos el inverso de *a* módulo *b*, que es *u* y recíprocamente, el inverso de *b* mod *a*, que es *v*.
- La función *inverso(a,m)*, que calcula el inverso multiplicativo de *a* módulo *m* si *a* y *m* son primos relativos.


In [17]:
import gmpy2
from gmpy2 import mpz
import sympy
import numpy as np

In [19]:
def algoritmo_euclides(a,b):
    if gmpy2.t_mod(a,b) == 0:
        return b
    else:
        return algoritmo_euclides(b, gmpy2.t_mod(a,b))

In [20]:
mcd=algoritmo_euclides(39,150)
print(mcd)

3


In [21]:
algoritmo_euclides(7,15)

mpz(1)

In [23]:
def algoritmo_euclides_extendido(a,b):
    """
    # Condición a>b, sino las cambiamos
    if b>a:
        aux=a
        a=b
        b=aux
    """
    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(gmpy2.c_mod(b,a), a)
        u=gmpy2.sub(y,(gmpy2.mul(gmpy2.c_div(b,a),x)))
        v=x
        
    return mcd, u, v

In [24]:
def inverso(a,m):
    result = algoritmo_euclides_extendido(a,m)
    # Comprobamos que el mcd es 1 para que exista inverso multiplicativo
    # En consecuencia, a y m determinan una función afín inyectiva
    if result[0] == 1:
        # Entonces devolvemos el coeficiente u (que acompaña a) de la Id. de Bézout
        inv=result[1]
        return inv
    else:
        print("Error")

In [25]:
inverso(51,23)

mpz(-9)

Las siguientes dos funciones: *read_input* y *read_output* se utilizarán a lo largo de todos los ejercicios de esta práctica para gestionar el comportamiento de entrada y salida requerido. 

- *read_input(i)* lee por la entrada estándar si el parámetro *i* es nulo. De lo contrario, abre el fichero pasado como parámetro.
- *read_output(o, cadena)* imprime el parámetro *cadena* por la entrada estándar si el parámetro *o* es nulo. De lo contrario, escribe  *cadena* en el fichero *o*, y si no existe, lo crea.

In [26]:
def read_input(i):
    # Primero tomamos el input de i o de la entrada estándar
    if i==0:
        cadena=input()
    else:
        file=open(i, "r")
        cadena=file.read()
        file.close()
    
    if len(cadena)<50:
        print("Cadena: {}".format(cadena))
    return cadena

In [27]:
def print_output(o,cadena):
    if o==0:
        print("Cadena: {}".format(cadena))
    else:
        file=open(o, "w")
        cadenaToStr = ' '.join([str(elem) for elem in cadena])
        file.write(cadenaToStr)
        file.close()

La siguiente función implementa el método afín.

La llamada a la función:

**afin {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i filein] [-o fileout]**

- -C el programa cifra
- -D el programa descifra
- -m tamaño del espacio de texto cifrado
- -a coeficiente multiplicativo de la función afín
- -b término constante de la función afín
- -i fichero de entrada
- -o fichero de salida

Como estamos trabajando con funciones de Python en celdas de Jupyter notebook, no con scripts, la llamada de la función se realiza en una celda así: afin(modo,m,a,b,i,o), indicando como modo *-C* o *-D*, *m*, *a*, *b* como enteros y *i*, *o* se introducen opcionalmente: si no se especifican, por defecto realiza la operación (entrada o salida) con la estándar y si se especifican, se trabaja con los ficheros proporcionados. 

Los parámetros *a* y *m* deben ser primos relativos, es la primera condición que verifica nuestra función. Si lo son, continúa con el algoritmo de cifrado o descifrado, según se haya especificado en la llamada. Además, *m* debe se la longitud de nuestro alfabeto.

El esquema de funcionamiento que sigue este método y los demás métodos de la práctica es:

1. Traducimos el input de caracteres a números enteros utilizando nuestro alfabeto de 52 elementos.
2. Aplicamos el mecanismo de cifrado o descifrado sobre la cadena numérica. En este caso el del cifrado afín.
3. Obtenemos una cadena cifrada numérica, que pasamos de números a caracteres usando nuestro alfabeto.


In [28]:
def afin(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            
            #Traducimos la cadena_cifrada numérica a caracteres
            resul=""
            for i in range(len(cadena_cifrada)):
                   resul=resul+alfabeto[cadena_cifrada[i]]
                    
            print_output(o,resul)
            
        elif modo=="-D":
            
            cadena_cifrada=read_input(i)
            
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena_cifrada:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
                    
            cadena_descifrada=[]
            cadena_texto=""
            #inv=inverso(m,a)
            inv=pow(a, -1, m)
            print(inv)
            for i in range(len(cadena_numerica)):
                cadena_numerica[i]=int(cadena_numerica[i])
                
            for k in cadena_numerica:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0:
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
                cadena_texto=cadena_texto+alfabeto[k_descifrado]
            
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [29]:
afin("-C",51,23,3,"cadena.txt","cadena_cifrada.txt")

Cadena: Hola Nueva York!


In [30]:
afin("-D",51,23,3, "cadena_cifrada.txt")

Cadena: W t b d H e S B d F t L D
20
Cadena: HolaNuevaYork


In [31]:
afin("-C",130,16,27)

16 y 130 no son primos relativos. Error


In [32]:
afin("-C",51,13,0)

Universidad Autonoma de Madrid
Cadena: Universidad Autonoma de Madrid
Cadena: LqcsbrEcNaNGfRDqDdaNbJaNrcN


In [33]:
afin("-D",51,13,0)

LqcsbrEcNaNGfRDqDdaNbJaNrcN
Cadena: LqcsbrEcNaNGfRDqDdaNbJaNrcN
4
Cadena: UniversidadAutonomadeMadrid


## 1.b Criptoanálisis del cifrado afín

Nuestro afin no trivial se basa en aumentar el tamaño de la clave. Hemos decidido cambiar el tamaño de la base y cifrarlo en bigramas. Lo que nos daría un espacio de 676.Con el afín normal, en nuestro caso, obteniamos uno de 52. 

Como se puede observar en la función fortaleza y en su ejecución el normal nos daría 128 claves y el no trivial 11545444563871328761349212098135488565445348609393477048015277366400000000. 



In [34]:
def fortaleza(m):
    z_m_inv=sympy.totient(m) #calculamos la funcion phi
    return gmpy2.mul((m),z_m_inv) #calculamos la fortaleza multiplicando Zm y Zm*


In [35]:
print(fortaleza(26**26+26))
#fortaleza(26) este seria el resultado si no aceptaramos mayúsculas
print(fortaleza(52))


11545444563871328761349212098135488565445348609393477048015277366400000000
1248


In [36]:
alfabeto='abcdefghijklmnopqrstuvwxyz'
digrama=([])
for i in alfabeto:
    for j in alfabeto:
        digrama.append(i+j)



El siguiente programa implementa el método afín no trivial.

Llamada a la función:

afin_no_trivial {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i filein] [-o fileout]

- -C el programa cifra
- -D el programa descifra
- -m tamaño del espacio de texto cifrado
- -a coeficiente multiplicativo de la función afín
- -b término constante de la función afín
- -i fichero de entrada
- -o fichero de salida

En este programa utilizamos un alfabeto de 26 elementos, las letras del abecedario en minúsculas


In [37]:
def afin_no_trivial(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
    digrama=([]) #generamos un vetor de di-gramas del alfabeto que hemos declarado arriba
    for k in alfabeto:
        for j in alfabeto:
            digrama.append(k+j)
    
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            #obtenemos el texto claro, si no se ha pasado por parámetro, se obtine de teclado
            cadena=read_input(i)
            #if i==0:
            #cadena=input()
            #cadena=read_input(i)
            #else:
            #    file=open(i, "r")
             #   cadena=file.read()
              #  file.close()
            
            #Traducimos los caracteres a números para poder operar
            cadena_numerica=[]
            j=0
            if len(cadena)%2==0:
                while j <(len(cadena)-1):
                    cadena_numerica.append(digrama.index(cadena[j]+cadena[j+1]))
                    j=j+2
            else:
                while j <(len(cadena)-2):
                    cadena_numerica.append(digrama.index(cadena[j]+cadena[j+1]))
                    j=j+2
                cadena_numerica.append(alfabeto.index(cadena[len(cadena)-1])) #como es impar, la última letra la ciframos aparte
            
            #utilizamos la función de cifrado
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            
            #pasamos el resultado a caracteres y lo guardamos como string
            resul=""
            for k in cadena_cifrada:
                resul=resul+digrama[k]
          
            #si no se ha pasado un fichero de salida por parámetro, imprimimos el resultado
            print_output(o,resul)
            
        elif modo=="-D":
            
            cadena_cifrada=read_input(i)
            #cadena_cifrada=input()
    
            cadena_descifrada=[]
            cadena_texto=""
            
             #obtenemos el texto claro, si no se ha pasado por parámetro, se obtine de teclado
            #if i==0:
             #    cadena_cifrada=input()
            #cadena=read_input(i)
            #else:
             #   file=open(i, "r")
             #   cadena_cifrada=file.read()
              #  file.close()
                
            #obtenemos el inverso en el modulo para poder utilizar la función de descifrado
            #ya hemos comprobado al principio que el inverso existe.
           
            #inv=inverso(a,m)
            inv=pow(a, -1, m)
            
            #pasamos el texto a un formato numérico para poder operar
            cadena_numerica=[]
            j=0
            if len(cadena_cifrada)%2==0:
                while j <(len(cadena_cifrada)-1):
                    cadena_numerica.append(digrama.index(cadena_cifrada[j]+cadena_cifrada[j+1]))
                    j=j+2
            else:
                while j <(len(cadena_cifrada)-2):
                    cadena_numerica.append(digrama.index(cadena_cifrada[j]+cadena_cifrada[j+1]))
                    j=j+2
                cadena_numerica.append(alfabeto.index(cadena_cifrada[len(cadena_cifrada)-1])) #como es impar, la última letra la ciframos aparte
            
            #desciframos el texto con la función de descifrado: 
            for k in cadena_numerica:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0: #ajustamos el modulo
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
            
            #pasamos el texto a caracteres
            if len(cadena_cifrada)%2==0:
                for k in cadena_descifrada:
                    cadena_texto=cadena_texto+digrama[k]
            else:       
                for k in range(len(cadena_descifrada)-1):
                    cadena_texto=cadena_texto+digrama[cadena_descifrada[k]]
                    
                cadena_texto=cadena_texto+alfabeto[cadena_descifrada[(len(cadena_descifrada)-1)]]
            
            #si no se ha pasado un archivo para guardar el resultado, se imprime por pantalla
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [41]:
afin_no_trivial("-C",701,23,3)

hola
Cadena: hola
Cadena: ltkm


In [42]:
afin_no_trivial("-D",701,23,3)

ltkm
Cadena: ltkm
Cadena: hola


In [ ]:
afin_no_trivial("-D",701,52,3)

El cifrado afin muy vulnerable a los ataques. Se rompe imediatamente con B,C,D y E. Con A hace falta un analisis de frecuencias (El análisis de frecuencia es el estudio de la frecuencia de letras o grupos
de letras en un texto cifrado).

#### Ejemplo de criptoánalisis afín

Hemos cifrado "antiaereo" con afin (modulo 51, a=13 y b=0), lo que nos ha dado la cadena "aqRcabrbD". 

En el ejemplo de abajo hemos guardado las tablas de frecuencia del castellano y el ingles y luego las hemos ordenado por mayor a menor. En este caso solo hemos utilizado la del castellano.

Hemos conseguido descifrarlo con la segunda hipotesís:
c1: la posición del elemento más utilizado de la cadena en el alfabeto.
c2: la posición del segundo elemento más utilizado de la cadena en el alfabeto.
t1: la posición del elemento más utilizado en el alfabeto.
t2: la posición del segundo elemento más utilizado en el alfabeto.

pos(c1)=pos(t1)* a+b 
pos(c2)=pos(t1)* a+b

Se puede resolver de dos formas:
1- restando las ecuaciones, lo que nos daría el resultado de: 
      pos(c1)-pos(c2)=(pos(t1)-pos(t2))* a -> 
      a=pos(c1)-pos(c2) inv((pos(t1)-pos(t2))
      b=pos(c1)-pos(t1)* a

2- Al introducir los datos conocidos, nos damos cuenta de que se puede simplificar y resolver casi directamente:
      pos(c1)=pos(t1)* a+b -> 1=4* a+b->a=1* inv(4)=13
      pos(c2)=pos(t1)* a+b -> 0=0* a+b ->b=0

En ambos casos el resultado da a=13, b=0. El cifrado esta roto.

In [43]:
from collections import Counter

cadena = "aqRcabrbD"
letters = Counter(cadena)

castellano=(['a', 11.96],['b', 0.92],['c', 2.92],['d', 6.87],['e', 16.78],['f', 0.52],['g', 0.73],
           ['h', 0.89],['i', 4.15],['j', 0.3],['k', 0.0],['l', 8.37],['m', 2.12],['n', 7.01],
           ['o', 8.69],['p', 2.77],['q', 1.53],['r', 4.94],['s', 7.88],['t', 3.31],['u', 4.80],
           ['v', 0.39],['w', 0.0],['x', 0.06],['y', 1.54],['z', 0.15])

ingles=(['a', 11.96],['b', 1.54],['c', 3.06],['d', 3.99],['e', 12.51],['f', 2.30],['g', 1.96],
        ['h', 0.89],['i', 7.26],['j', 0.16],['k', 0.67],['l', 4.14],['m', 2.53],['n', 7.09],
       ['o', 7.60],['p', 2.0],['q', 0.11],['r', 6.12],['s', 6,54],['t', 9.25],['u', 2.71],
       ['v', 0.99],['w', 1.92],['x', 1.92],['y', 1.73],['z', 0.19])

letras=sorted(letters, reverse=True)
castellano_ord=sorted(castellano, key=lambda letra: letra[1], reverse=True)
ingles_ord=sorted(ingles, key=lambda letra: letra[1], reverse=True)


print("segunda hipotesis")
c1=alfabeto.index("b")
c2=alfabeto.index("a")
t1=alfabeto.index(castellano_ord[0][0])
t2=alfabeto.index(castellano_ord[1][0])

#pos(c1)=pos(t1)*a+b -> 1=4*a+b->a=1*inv(4)
#-
#pos(c2)=pos(t1)*a+b -> 0=0*a+b ->b=0

#pos(c1)-pos(c2)=(pos(t1)-pos(t2))*a

#1=(-4)*
#0-1=(4-0)*a

#a=1*inv(4)

#comprobamos que hemos resuelto bien la ecuación
a=1*pow(4,-1,51)
#comprobamos que sean co-primos
if algoritmo_euclides(a,51)==1:
        b=c1-int(a)*t1
        print("es correcto, antiaereo se cifro con 13 y 0")
        #print(afin("-D", 51, a, b))
        
              
    

segunda hipotesis
es correcto, antiaereo se cifro con 13 y 0


## 2. Sustitución polialfabeto

## 2.a Método de Hill

El siguiente programa implementa el método hill.

Llamada a la función:

hill {-C|-D} {-m |Zm|} {-n NK} {-k f ileK} [-i f ilein] [-o f ileout]

Los parámetros introducidos en este caso son:
- m cardinalidad de Zm
- n dimensión de la matriz de transformación
- k fichero que contiene la matriz de transformación

In [44]:
import numpy as np
import os
import math
import copy

Funcion que cálcula el determinante de una matriz

In [45]:
def determinante(matriz):
   
    if len(matriz)==2 and len(matriz[0])==2:
        #calculamos el determinante
        det=matriz[0][0]*matriz[1][1]-(matriz[1][0]*matriz[0][1])
       
        return det
    else:
        suma=0
        for i in range(len(matriz)): #calculamos el determinante por cofactores
            maux=copy.deepcopy(matriz)
            maux.remove(matriz[0]) #eliminamos la primera fila
            for j in range(len(maux)):
                maux[j]=maux[j][0:i]+maux[j][i+1:]
                
         
            suma= suma+ (-1)**((i+j)%2)*matriz[0][i]*determinante(maux)
            
        return suma
        

In [46]:
#comprobación de la función
matriz = [[1,2,3], [3,4,5], [1,4,3]]
print(determinante(matriz))
matriz = [[11,8], [3,7]]
print(determinante(matriz))

-8
53


Función que cálcula el adjunto de una matriz

In [47]:
def adjunto(matriz):
    adjunto=np.zeros(np.shape(matriz))
    if len(matriz)==2 and len(matriz[0])==2:
         #calculamos el adjunto
        adjunto[0][0]=matriz[1][1]
        adjunto[0][1]=-matriz[0][1]
        adjunto[1][0]=-matriz[1][0]
        adjunto[1][1]=matriz[0][0]
        
        return adjunto
    else:
        
        for i in range(len(matriz)):
            maux=copy.deepcopy(matriz)
            for j in range(len(matriz)):
             
                maux=np.delete(matriz,i,0)
                aux=np.delete(maux,j,1)
                auxi=aux.tolist()
                #la matriz de cofactores transpuesta es el djunto
                adjunto[j][i]=(-1)**((i+j)%2)*determinante(auxi)
            
                
        return adjunto

In [48]:
#comprobación de la función
matriz = [[11,8], [3,7]]
print(adjunto(matriz))

[[ 7. -8.]
 [-3. 11.]]


Función que cálcula la inversa de una matriz

In [49]:
def inversa(matriz,modulo):
    inversa=np.zeros(np.shape(matriz))
    det=determinante(matriz)%modulo
    if det !=0:
        adj=adjunto(matriz)%modulo
        for i in range(len(matriz)):
            for j in range(len(matriz[i])):
                inversa[i][j]=(adj[i][j]/det)#%modulo
                
    return inversa%modulo #esto puede que no sea necesario porque ya estamos en matemática modular

In [50]:
#comprobación de la función
matriz = [[11,8], [3,7]]
print(inversa(matriz,26))

[[ 7. 18.]
 [23. 11.]]


In [51]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
np.delete(arr, 1, 0)

array([[ 1,  2,  3,  4],
       [ 9, 10, 11, 12]])

In [52]:
matriz = [[1, 2, 4], [3,5,4], [1]]

# Padding a la matriz
n=3
print(matriz)
print(len(matriz[n-1]))
k=len(matriz[n-1])
if k<n:
    padding=[0]*(n)
    for i in range(0,k):
        padding[i]=matriz[n-1][i]
    matriz[n-1]=padding
  
print(matriz)
matriz=np.array(matriz)
matriz%3

[[1, 2, 4], [3, 5, 4], [1]]
1
[[1, 2, 4], [3, 5, 4], [1, 0, 0]]


array([[1, 2, 1],
       [0, 2, 1],
       [1, 0, 0]], dtype=int32)

In [53]:
print(matriz[1])

[3 5 4]


Función de cifrado del algoritmo.

Parámetros:
- matriz_numerica: el texto a cifrar en formato matriz de numeros
- matriz: matriz de transformación
- mod: modulo en el que trabajamos
- n: dimensión 

In [54]:
def cifrar(matriz_numerica, matriz,mod,n):
    matriz_cifrada=np.zeros((n,n))
    
    for i in range(len(matriz_numerica)):
        cadena_cifrada= (np.dot(matriz_numerica[i],matriz))%mod
        matriz_cifrada[i]=cadena_cifrada

    return matriz_cifrada  

Función de descifrado del algoritmo.

Parámetros:

- matriz_cifrada: el cifrado a descifrar en formato matriz de numeros
- matriz: matriz de transformación
- mod: modulo en el que trabajamos
- n: dimensión

In [55]:
def descifrar(matriz_cifrada, matriz,mod,n):
    matriz_descifrada=np.zeros((n,n))
    inv=inversa(matriz,mod)
 
    for i in range(len(matriz_cifrada)):
        cadena_descifrada= (np.dot(matriz_cifrada[i],inv))%mod
        matriz_descifrada[i]=cadena_descifrada

    return matriz_descifrada  

In [56]:
def hill(modo,mod,n,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
  
    #leemos la matriz de transformación del archivo y la guardamos
    with open(k,'r') as f:
        datos = ''.join(f.readlines()).replace('\n',';')
    matriz = np.matrix(datos).tolist()
    f.close()
   
    #cálculamos el determinante de la matriz
    det=np.linalg.det(matriz)
    #comprobamos que la matriz K tiene una función biyectiva
    if algoritmo_euclides(int(det),mod)==1:
       
        if modo=="-C":
           
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
                    
            # Dividimos en bloques de n elementos el texto
            # Si m no es múltiplo de n se añade padding
            m=len(cadena_numerica)/n
            max=len(cadena_numerica)
            matriz_numerica=np.zeros((math.ceil(m),n))
         
            pos=0
            for i in range(math.ceil(m)):
                for j in range(n):
                    if pos<max:
                        matriz_numerica[i][j]=cadena_numerica[pos]
                        pos=pos+1
                        
            #ciframos cadena a cadena y lo guardamos en un matriz           
            matriz_cifrada=cifrar(matriz_numerica,matriz,mod,n)
            print_output(o,matriz_cifrada)
           
            
        elif modo=="-D":
            if i==0:
                cadena_cifrada=input()
                mat_cifrada = np.matrix(cadena_cifrada).tolist()
                matriz_cifrada= np.reshape(mat_cifrada, (n,n))
               
                        
            else:
                
                with open(i,'r') as f:
                    datos = ''.join(f.readlines()).replace('\n',';')     
                f.close()
                
                matriz_cifrada = np.matrix(datos).tolist() 
        
           
            matriz_descifrada=descifrar(matriz_cifrada, matriz,mod,n)
            
            cadena_num=[]*n*n
            for i in range(len(matriz_descifrada)):
                for j in range(len(matriz_descifrada[i])):
                    
                    if matriz_descifrada[i][j]<0:
                        matriz_descifrada[i][j]=mod+matriz_descifrada[i][j]
        
                    cadena_num.append(alfabeto[int(matriz_descifrada[i][j])])
            print_output(o,cadena_num)
            
    else:
        print("{} y {} no son primos relativos. Error".format(det,mod))

In [57]:
k = [[11,8], [3,7]]
hill("-D",26, 2,"matriz_k.txt","matriz_cifrada.txt" ,0)

Cadena: ['h', 'o', 'l', 'a']


In [58]:
hill("-C",26,2, "matriz_k.txt", 0,0 )


Cadena: 
Cadena: [[0. 0.]
 [0. 0.]]


In [59]:
cadena_numerica=[8,15,12,1]
matriz = [[11,8], [3,7]]
n=2

m=len(cadena_numerica)/n
max=len(cadena_numerica)
matriz_numerica=np.zeros((math.ceil(m),n))

pos=0
for i in range(math.ceil(m)):

    for j in range(n):
        
        if pos<max:
           
            matriz_numerica[i][j]=cadena_numerica[pos]
            pos=pos+1
            
print("matriz_numerica")
print(matriz_numerica)   

matriz_cifrada=cifrar(matriz_numerica, matriz,26,n)
print("matriz cifrada")
print(cifrar(matriz_numerica, matriz,26,n))

print(descifrar(matriz_cifrada, matriz,26,n))
print("matriz descifrada")
matriz_descifrada=descifrar(matriz_cifrada, matriz,26,n)
alfabeto='abcdefghijklmnopqrstuvwxyz'
cadena_num=[]*n*n
for i in range(len(matriz_descifrada)):
    for j in range(len(matriz_descifrada[i])):
        if matriz_descifrada[i][j]<0:
            matriz_descifrada[i][j]=26+matriz_descifrada[i][j]
        cadena_num.append(alfabeto[int(matriz_descifrada[i][j])-1])
print(cadena_num)

matriz_numerica
[[ 8. 15.]
 [12.  1.]]
matriz cifrada
[[ 3. 13.]
 [ 5. 25.]]
[[ 8. 15.]
 [12.  1.]]
matriz descifrada
['h', 'o', 'l', 'a']


In [60]:
with open('matriz_k.txt','r') as f:
    datos = ''.join(f.readlines()).replace('\n',';')

matriz = np.matrix(datos)
print(matriz)

[[11  8]
 [ 3  7]]


## 2.b Método de Vigenere

El siguiente programa implementa el método de Vigenere.

Llamada a la función: 

vigenere {-C|-D} {-k clave} [-i filein] [-o fileout]

El parámetro *k* es cadena de caracteres usada como clave. Consideramos que la clave está formada por caracteres de nuestro alfabeto. Puede ser una frase, ya que se eliminarán los espacios. Al igual que hacemos con el input, la clave se traducirá de caracteres a números. 

Como el cifrado de Vigenere es un cifrado de bloques, entonces dividimos el input en bloques de n (longitud de la clave) elementos. Si la longitud del input no es múltiplo de la longitud de la clave, añadimos padding de ceros al final del último bloque para que todos los bloques tengan n elementos. Tras este proceso hemos obtenido una matriz de n columnas, en la que cada fila es un bloque.

Realizamos el cifrado de Vigenere a la matriz. Ciframos bloque a bloque recorriendo las filas de la matriz: al elemento i-ésimo del bloque se le suma el elemento i-ésimo de la clave y al resultado se le aplica el módulo de la longitud del alfabeto. Esta operación se realiza para cada elemento del bloque ( i de 1 a n).  Como resultado, obtenemos una matriz de bloques cifrados. A continuación concatenamos esta matriz para obtener la cadena cifrada resultante.

A la hora de descifrar, el procesamiento es el mismo pero la operación que se realiza a la hora de descifrar bloque a bloque es diferente, naturalmente. Para cada elemento i-ésimo del bloque, se resta el elemento i-ésimo de la clave y al resultado se le aplica el módulo de la longitud del alfabeto.


In [61]:
def vigenere(modo,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    base=len(alfabeto)
    
    #Traducimos la clave de caracteres a números
    k_numerica=[]
    for j in k:
        if j in alfabeto: 
            k_numerica.append(alfabeto.index(j))
    n=len(k_numerica)
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Dividimos en bloques de n elementos el input
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_numerica)/n
        maxi=len(cadena_numerica)
        matriz_numerica=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_numerica[i][j]=cadena_numerica[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que cifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_numerica.shape[0]
        elementos=matriz_numerica.shape[1]
        matriz_cifrada=np.zeros((filas,elementos))
        for i in range(filas):
            for j in range(elementos):
                matriz_cifrada[i][j]=(matriz_numerica[i][j]+k_numerica[j])%base
        
        cadena_cifrada=np.concatenate(matriz_cifrada)
        resul=""
        for i in cadena_cifrada:
            resul=resul+alfabeto[int(i)]
        print_output(o,resul)
    elif modo=="-D":
        cadena_cifrada_texto=read_input(i)
        cadena_cifrada=[]
        for k in cadena_cifrada_texto:
            if k in alfabeto: 
                cadena_cifrada.append(alfabeto.index(k))
        # Dividimos en bloques de n elementos el texto cifrado
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_cifrada)/n
        maxi=len(cadena_cifrada)
        matriz_cifrada=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_cifrada[i][j]=cadena_cifrada[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que descifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_cifrada.shape[0]
        elementos=matriz_cifrada.shape[1]
        matriz_descifrada=np.zeros((filas,elementos))
        for i in range(filas):
            for j in range(elementos):
                matriz_descifrada[i][j]=(matriz_cifrada[i][j]-k_numerica[j])%base
        cadena_descifrada=np.concatenate(matriz_descifrada)
        cadena_texto=[]
        for i in range(len(cadena_descifrada)):
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [62]:
vigenere("-C", "clave")

Universidad Autonoma de Madrid
Cadena: Universidad Autonoma de Madrid
Cadena: WyiQitDiyefLuOspzmvhgXayvkoave


In [63]:
vigenere("-D", "clave")

WyiQitDiyefLuOspzmvhgXayvkoave
Cadena: WyiQitDiyefLuOspzmvhgXayvkoave
Cadena: ['U', 'n', 'i', 'v', 'e', 'r', 's', 'i', 'd', 'a', 'd', 'A', 'u', 't', 'o', 'n', 'o', 'm', 'a', 'd', 'e', 'M', 'a', 'd', 'r', 'i', 'd', 'a', 'a', 'a']


In [98]:
vigenere("-C", "probamos con otra clave mas larga")

Universidad Autonoma de Madrid
Cadena: Universidad Autonoma de Madrid
Cadena: jEwweDGAfoqONKopzmvhqMsorzja


In [99]:
vigenere("-D", "probamos con otra clave mas larga")

jEwweDGAfoqONKopzmvhqMsorzja
Cadena: jEwweDGAfoqONKopzmvhqMsorzja
Cadena: ['U', 'n', 'i', 'v', 'e', 'r', 's', 'i', 'd', 'a', 'd', 'A', 'u', 't', 'o', 'n', 'o', 'm', 'a', 'd', 'e', 'M', 'a', 'd', 'r', 'i', 'd', 'a']


In [ ]:
vigenere("-C", "clave", "texto_vigenere.txt", "resultado_vigenere.txt")

In [ ]:
vigenere("-D", "clave", "resultado_vigenere.txt", "descifrado_vigenere.txt")

También probamos el cifrado de Vigenere para cifrar el Quijote con la clave "En un lugar de la Mancha, de cuyo nombre no quiero acordarme".

In [ ]:
vigenere("-C","En un lugar de la Mancha, de cuyo nombre no quiero acordarme", "quijote.txt", "quijote_cifradoVigenere.txt")

In [ ]:
vigenere("-D", "En un lugar de la Mancha, de cuyo nombre no quiero acordarme", "quijote_cifradoVigenere.txt", "quijote_descifradoVigenere.txt")

## 2.c Criptoanálisis del cifrado de Vigenere

El siguiente programa implementa el indice de coincidencia.

Llamada a la función: 
IC {-l Ngrama} [-i filein] [-o fileout]

-l longitud de n-grama buscado


In [64]:
El siguiente programa implementa el test de kasiski.

Llamada a la función: 
kasiski {-l Ngrama} [-i f ilein] [-o f ileout]
-l longitud de n-grama buscado

SyntaxError: invalid syntax (Temp/ipykernel_6656/2227801207.py, line 1)

In [65]:
def calcular_divisores(n):
    lista = []
    for i in range(2,n):
        if n % i == 0:
            lista.append(i)
    return lista

In [66]:
n=25
print(calcular_divisores(n))

[5]


In [67]:
def obtener_tuplas(lista):
    res = {}
    freq =[]
    cont = 0
    i = 0
    while i < len(lista): 
        elt= lista[i:i+3] # Cogemos al menos 3 caracteres como tamaño de la tupla
        tam = len(elt)
        if tam == 3: #tiene que ser 3, si no estamos al final de la lista
            for j in range(i+1,len(lista)): #Find further in the list for the same pattern
                if lista[i:i+tam] == lista[j:j+tam]: #Si coinciden, seguimos comprobando
                    while lista[i:i+tam] == lista[j:j+tam]:
                        tam = tam + 1
                    tam = tam -1
                    elt = lista[i:i+tam] #Ahora tenemos una tupla 
                    dist = j - i #calculamos la distancia
                    freq.extend(calcular_divisores(dist)) #Añadimos los divisores a la lista 
                    print ("%s\ti:%s\tj:%s\tdiff:%s\t\tDivisors:%s" % (elt,i,j, dist,calcular_divisores(dist))) #Print information about the tuple (can be deleted)
                    cont = cont +1
                    j = j + tam + 1
            i = i + tam -3 +1
        else:
            i = i + 1
    return cont, freq

In [68]:
lista="jzlvuwptvpnwaHedlpvvcnoIxcCtzuwptJhqGawmgyyLygwaHyutcvwkruzwqyaIhqnoHscytzw"
cont, freq=obtener_tuplas(lista)
print(cont)
print(freq)

uwpt	i:4	j:29	diff:25		Divisors:[5]
waH	i:11	j:46	diff:35		Divisors:[5, 7]
2
[5, 5, 7]


In [69]:
def contDiv(lista): # devuelve una lista con (caracter_decimal, occ) 
    d={}
    for elt in lista:
        if d.__contains__(elt): #contamos los divisores de cada uno
            d[elt] += 1
        else:
            d[elt] = 1
    return sorted(d.items(),key=lambda x: x[1], reverse=True) #ordenamos la lista en orden descendente

In [70]:
def dividir_lista(key,lista): #ahora que sabemos la cardinalidad de la llave, dividimos la lista en ese modulo
    dic = {}
    for elem in range(key): #hacemos sublistas de key
        dic[elem] = []
        
    i = 0
    for j in range (len(lista)):
        if i == key: #si el inidice es igual a key hemos llegado al final de la sublista
            i = 0 #para la proxima sublista
        dic[i].append(lista[j])
        i = i + 1
    return dic

In [71]:
lista="jzlvuwptvpnwaHedlpvvcnoIxcCtzuwptJhqGawmgyyLygwaHyutcvwkruzwqyaIhqnoHscytzw"

key=5
#dividir_lista(key, lista)

In [72]:
def descifrar_v(lista,dist):
    alfabeto="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lista1=list()
    for elem in lista:
        valor = alfabeto.index(elem) - int(dist)
        if valor < 0:
            lista1.append(256 + (valor % -256))
        else:
            lista1.append(valor)
    return lista1

In [73]:
lista=input()

cont,freq=obtener_tuplas(lista)
l=contOcc(freq)
res = dividir_lista(l[i][0], lista) 
res=contOcc(freq)
occ = contOcc(res[i]) 
print(res[1])
print(occ[0][0])
print(lista[0])
print(l)
descifrar_v(res[0],occ[0][0])

NameError: name 'contOcc' is not defined

In [74]:
def recrear_lista(dic):
    i = 0
    output = []
    try:
        while 1:
            for l in dic.values():
                output.append(l[i])
            i = i + 1
    except:
        pass
    return output

In [75]:
def criptoanalisis_vigenere(lista):
        alfabeto="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        #cadena=input() #obtenemos la cadena a descifrar
        #lista=[]
        #for elem in cadena:
            #lista.append(alfabeto.index(elem)) #pasamos las letras a numeros
        
        #buscamos las tuplas del texto y la distancia entre ellas
        cont,freq=obtener_tuplas(lista)
        lista_modulos_posibles=contDiv(freq) #devuelve un lista con (divisor, #divisor)
        #la lista_modulos_posibles esta ordenada del que más aparece al que menos, asique elegimos el primer elemento
        
        res = dividir_lista(lista_modulos_posibles[0][0], lista) #We consider in this exemple a key length of 10 and l the original list
        #tenemos la lista original dividida en lista_modulos_posibles[0][0] elementos
        print(len(res))
        for i in range(lista_modulos_posibles[0][0]): #por cada sublista
            
            divisores = contDiv(res[i]) # Hacemos un analisis de frecuecia
            shift = (alfabeto.index(divisores[0][0]) - 32) % 256 # Consider the most frequent element of being a space(32 in decimal)
           
            print ("Frequency analysis for the index: %s\tshift:%s\n%s\n" % (i,shift,occ)) #Print informations (can be deleted)
            res[i] = descifrar_v(res[i],shift) #Intnemos descifrarlo 

        final = recrear_lista(res) #Once we have processed all sub-list recreate a list with all the sub-lists.
        print(final)
        print(len(alfabeto))
        resul=""
        for elem in final:
            resul=resul+alfabeto[elem-1]
        print(resul)
        #print (''.join([chr(x) for x in final])) #Print the result

In [76]:
lista="jzlvuwptvpnwaHedlpvvcnoIxcCtzuwptJhqGawmgyyLygwaHyutcvwkruzwqyaIhqnoHscytzw"
lista1="jzlviuEawerpnNepoozroFcCeunoNeuDowvgEoysgycJwcDqPipztDippmPgjzszrvtdJgqxozwvpeEitnixmqEoOenxeIxgtnOitxiIedweLygyoxspDiBsfpsxmhCaMtgCoLygllHipzsxsoAiGesFevpizeNenrove"

cont, freq=obtener_tuplas(lista)
print(cont)
print(freq)
criptoanalisis_vigenere(lista1)

uwpt	i:4	j:29	diff:25		Divisors:[5]
waH	i:11	j:46	diff:35		Divisors:[5, 7]
2
[5, 5, 7]
ipz	i:49	j:139	diff:90		Divisors:[2, 3, 5, 6, 9, 10, 15, 18, 30, 45]
Lyg	i:108	j:133	diff:25		Divisors:[5]
5


NameError: name 'occ' is not defined

## 3. Cifrado de flujo

El siguiente programa implementa el cifrado de flujo.

Llamada a la función: 

flujo {-C|-D} {-m clave} {-n tamaño de la secuencia de claves} [-i filein] [-o fileout]

Este cifrado es parecido al cifrado de Vigenere, de hecho, hemos visto que el cifrado de Vigenere se puede entender como un caso particular del cifrado de flujo. Su parecido reside en que el cifrado se realiza elemento a elemento operando con el elemento de la clave correspondiente. La diferencia es que las claves del cifrado de flujo pertenecen a una secuencia cifrante generada de manera aleatoria. Por lo tanto, para la implementación del cifrado de flujo es necesario programar un generador de una secuencia de números aleatorios, es decir, la secuencia cifrante. Hemos realizado un cifrado síncrono de flujo ya que el flujo de claves se codifica a partir de una clave que es independiente del texto original.
Nuestro funcion *generador_aleatorio* genera la secuencia cifrante.

Para cifrar, este método toma para cada elemento, un número aleatorio de la secuencia y los opera con la aplicación XOR lógica. Obtenemos una la cadena cifrada y expresada en binario, ya que actualmente los cifrados de flujo normalmente se expresan en alfabeto binario.


La fortaleza de este cifrado depende directamente del generador de la secuencia aleatoria. En nuestro caso **FALTA**

In [77]:
def rec_fib(n):
    if n > 1:
        return rec_fib(n-1) + rec_fib(n-2)
    return n

In [78]:
# Generador de secuencia aleatoria
def generador_aleatorio(m,cont):
    k=(rec_fib(m)%m)*m*cont
    return k

In [79]:
# Ejemplo de secuencia cifrante de 5 elementos para clave 14
m=14
for i in range(5):
    k=generador_aleatorio(m,i)
    print(k)

0
182
364
546
728


In [80]:
# m es la clave
# n es el tamaño de la secuencia de claves
def flujo(modo,m,n,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Ciframos carácter a carácter
        cadena_cifrada=[]
        count=0
        for i in cadena_numerica:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_cifrada.append(int(bin(i^k)[2:]))
            count=count+1
        print_output(o,cadena_cifrada)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        cadena_descifrada=[]
        cadena_texto=[]
        for i in range(len(cadena_cifrada)):
            cadena_cifrada[i]=int(cadena_cifrada[i],2)
        count=0
        for i in cadena_cifrada:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_descifrada.append((i^k))
            count=count+1
        for i in range(len(cadena_descifrada)):
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [81]:
flujo("-C", 4,2)

Hola
Cadena: Hola
Cadena: [100001, 10, 1011, 1100]


In [82]:
flujo("-D", 4,2)

100001 10 1011 1100
Cadena: 100001 10 1011 1100
Cadena: ['H', 'o', 'l', 'a']


In [ ]:
flujo("-C", 10,20, "cadena.txt", "cadena_cifradaFlujo.txt")

In [ ]:
flujo("-D", 10, 20, "cadena_cifradaFlujo.txt", "cadena_descifradaFlujo.txt")

A continuación, aplicamos el cifrado de flujo al libro del Quijote.

In [ ]:
flujo("-C", 8, 500, "quijote.txt", "quijote_cifradoFlujo.txt")

In [ ]:
flujo("-D", 8, 500, "quijote_cifradoFlujo.txt", "quijote_descifradoFlujo.txt")

## 4. Producto de criptosistemas de permutación

In [95]:
# k1: vector de m elementos que constituye la clave para el cifrado de permutación por filas
# k2: vector de n elementos que constituye la clave para el cifrado de permutación por columnas
def permutacion(modo,k1,k2,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))

        # Matriz numerica es la matriz m x n 
        m=len(k1)
        n=len(k2)
        maxi=len(cadena_numerica)
        matriz_numerica=np.zeros((m,n))
         
        pos=0
        for i in range(m):
            for j in range(n):
                if pos<maxi:
                    matriz_numerica[i][j]=cadena_numerica[pos]
                    pos=pos+1
        #print(matriz_numerica)
        
        # Cifrado de permutación por filas
        matriz_cifrada1=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_cifrada1[i][j]=matriz_numerica[k1[i]][j]
        #print(matriz_cifrada1)
                
        # Cifrado de permutación por columnas
        matriz_cifrada2=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_cifrada2[i][j]=matriz_cifrada1[i][k2[j]]
                
        #print(matriz_cifrada2)
        cadena_cifrada=np.concatenate(matriz_cifrada2)
        resul=""
        for i in cadena_cifrada:
            resul=resul+alfabeto[int(i)]
      
        print_output(o, resul)
    elif modo=="-D":
        cadena_cifrada_texto=read_input(i)
        cadena_cifrada=[]
        for k in cadena_cifrada_texto:
            if k in alfabeto: 
                cadena_cifrada.append(alfabeto.index(k))
        
        
         # Matriz numerica es la matriz m x n 
        m=len(k1)
        n=len(k2)
        maxi=len(cadena_cifrada)
        matriz_cifrada=np.zeros((m,n))
         
        pos=0
        for i in range(m):
            for j in range(n):
                if pos<maxi:
                    matriz_cifrada[i][j]=cadena_cifrada[pos]
                    pos=pos+1
        #print(matriz_cifrada)
        
        # Desciframos cifrado de permutación por columnas
        matriz_descifrada2=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_descifrada2[i][k2[j]]=matriz_cifrada[i][j]
        
        #print(matriz_descifrada2)
        
        # Desciframos cifrado de permutación por filas
        matriz_descifrada1=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_descifrada1[k1[i]][j]=matriz_descifrada2[i][j]
        #print(matriz_descifrada1)
        
        cadena_descifrada=np.concatenate(matriz_descifrada1)
        cadena_texto=[]
        for i in range(len(cadena_descifrada)):
            #cadena_cifrada[i]=int(cadena_cifrada[i])
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])

        print_output(o, cadena_texto)

In [96]:
k1=[3,2,4,1,0]
k2=[1,3,2,0]
permutacion("-C", k1, k2, "cadena.txt", "cadena_cifradaPermutacion.txt")

Cadena: Hola Nueva York!


In [97]:
permutacion("-D", k1, k2, "cadena_cifradaPermutacion.txt", "cadena_descifradaPermutacion.txt")

Cadena: a a a k Y r o a a a a a u v e N o a l H
